# ===Dataset  on  Airbnb

![Illustration du cleaning](https://assets.technologyadvice.com/uploads/2022/06/Data-Cleaning-scaled.jpeg)

In [5]:
# commencons  par  importer  toutes  les  librairies dont  nous aurons  besoins  pour  le  traitement  de  ce dataset
import  pandas as  pd 
import  numpy as  np
import matplotlib.pyplot as  plt
from matplotlib.collections import LineCollection
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
import scipy.stats as st
from scipy.stats import ks_2samp
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import statsmodels.formula.api as smf
import statsmodels.api as sm
import math as ma
from collections import Counter

#Une fois  les  modules  importés  nous allons  charger  les données  

## Chargement de  la  table  (dataset) 

In [24]:
df = pd.read_csv("data/listing_paris.csv")
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91031 entries, 0 to 91030
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              91031 non-null  int64  
 1   name                            91031 non-null  object 
 2   host_id                         91031 non-null  int64  
 3   host_name                       91022 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   91031 non-null  object 
 6   latitude                        91031 non-null  float64
 7   longitude                       91031 non-null  float64
 8   room_type                       91031 non-null  object 
 9   price                           60093 non-null  float64
 10  minimum_nights                  91031 non-null  int64  
 11  number_of_reviews               91031 non-null  int64  
 12  last_review                     

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,5396,Your perfect Paris studio on Île Saint-Louis,7903,Borzou,NaN,Hôtel-de-Ville,48.85247,2.35835,Entire home/apt,88.0,3,425,2024-11-23,2.26,1,77,51,7510402838018
1,7964,Sunny apartment with balcony,22155,Anaïs,NaN,Opéra,48.87417,2.34245,Entire home/apt,NaN,7,5,2015-09-14,0.03,1,0,0,7510903576564
2,9952,Paris petit coin douillet,33534,Elisabeth,NaN,Popincourt,48.86238,2.36957,Entire home/apt,150.0,4,52,2024-09-24,0.36,1,250,3,7511101582862
3,11487,"Heart of Paris, brand new aparment.",42666,Brigitte,NaN,Popincourt,48.86351,2.37110,Entire home/apt,80.0,30,13,2024-09-27,0.09,2,246,5,"Available with a mobility lease only (""bail mo..."
4,12452,Voltaire Charm,48733,Irene,NaN,Popincourt,48.85974,2.37932,Entire home/apt,201.0,1,62,2024-10-11,0.77,1,34,4,7511102600669


## Les données
Le fichier contient _91031_ observations avec *18* colonnes. Les premières observations :
* **Données géographiques** (latitude/longitude, quartiers)
* **Informations sur les hôtes** (host_id, host_name)
* **Caractéristiques des logements** (type, prix, nombre de chambres)
* **Métriques d'activité** (nombre d'avis, disponibilité)
### Observations Initiales

**Valeurs manquantes** : Plusieurs colonnes ont des valeurs manquantes, notamment price, last_review, reviews_per_month, et license.
**Types de données** : Les colonnes sont de types variés, y compris des entiers, des flottants et des chaînes de caractères.
**Données géographiques** : Les colonnes _latitude_ et _longitude_ peuvent être utilisées pour des analyses géospatiales.

## Prochaines Étapes

1. **Nettoyage des données** : Traiter les valeurs manquantes et nettoyer les données textuelles.
2. **Analyse exploratoire** : Explorer les données pour identifier des tendances et des relations.
3. **Visualisation** : Créer des graphiques pour mieux comprendre les données.

# Nettoyage des données

In [26]:
# Estimons et affichons  les valeurs  manquantes  
missing_data = df.isnull().sum() * 100 / len(df)
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)

for column, percentage in missing_data.items():
    print(f"{column}: {percentage:.2f}%")


neighbourhood_group: 100.00%
price: 33.99%
last_review: 26.21%
reviews_per_month: 26.21%
license: 20.64%
host_name: 0.01%


### Stratégies de  nettoyage 
Il y'a 6 colonnes ou nous constatons des valeurs  manquantes .
- Nous alllons  simplement supprimer  la colonne neighbourhood_group
- C'est une variable  importante que  nous allons traiter soit par imputation  soit  par regression  linéaire
- last_review et reviews_per_month : Environ 26% de valeurs manquantes. Nous pourrions :Imputer les valeurs manquantes avec des valeurs statistiques (moyenne, médiane) ou utiliser des techniques de régression.
- host_name : Très peu de valeurs manquantes. Nous pourrions simplement les remplir avec une valeur par défaut comme "Unknown".


# Appliquons  nos  actions  retenues  pour  nettoyer  ces  variables  
# Supprimer la colonne 'license'
df.drop(columns=['license'], inplace=True)

In [31]:
# Imputation des valeurs manquantes pour price
# Nous allons imputer les valeurs manquantes de la colonne price avec la médiane des prix disponibles.
# Imputer les valeurs manquantes de 'price' avec la médiane
df['price'].fillna(df['price'].median(), inplace=True)

/var/folders/xx/28bry1x90y31qtf30d895jzc0000gn/T/ipykernel_5728/4152222480.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price'].fillna(df['price'].median(), inplace=True)


In [35]:
# Imputation des valeurs manquantes pour reviews_per_month avec la moyenne
df['reviews_per_month'].fillna(df['reviews_per_month'].mean(), inplace=True)

/var/folders/xx/28bry1x90y31qtf30d895jzc0000gn/T/ipykernel_5728/3138743163.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['reviews_per_month'].fillna(df['reviews_per_month'].mean(), inplace=True)


In [37]:
# Remplissage des valeurs manquantes de last_review avec une date par défaut :
df['last_review'] = df['last_review'].fillna('2023-01-01')
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

In [39]:
# Remplissage des valeurs manquantes de host_name avec 'Unknown' :
df['host_name'] = df['host_name'].fillna('Unknown')
# Cette  technique  est  bien  car  elle  a  l'avantage  de  respecter  l'anonymisation du  jeu de donnée

In [51]:
# Supprimons  la colonne  Neighbourhood group  qui n'a  aucun  enregistrement  
df.drop(columns=['neighbourhood_group'], inplace=True)

In [55]:
# Suppression des doublons s'il  en existait  
df = df.drop_duplicates(subset=['id'])

In [59]:
# Nettoyage des textes
df['name'] = df['name'].str.strip()

# Cette opération de nettoyage vise à standardiser le texte en supprimant les espaces superflus au début et à la fin des chaînes de caractères dans la colonne name
# Supprimer les espaces multiples internes
df['name'] = df['name'].str.replace(r'\s+', ' ', regex=True)

# Uniformiser la casse (majuscules/minuscules)
df['name'] = df['name'].str.title()  # Première lettre en majuscule

### Nettoyage du texte
Il est  important de bien  nettoyer les chaines de caracteres  car je sais  par avance que  je  ferai  du  NLP  sur  ce  jeu  de donnée donc  c'est  important  de  bien  le nettoyer
Pourquoi c'est critique pour la suite ?
Des données textuelles propres sont essentielles pour :
* **L'analyse NLP** (extraire des mots-clés des descriptions).
* **Le machine learning** (éviter des doublons artificiels).
* **Les visualisations** (légendes propres dans les graphiques).

In [61]:
# Maintenant affichons  notre  table  nettoyée  pour constater  les  actions  realisées 
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91031 entries, 0 to 91030
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              91031 non-null  int64         
 1   name                            91031 non-null  object        
 2   host_id                         91031 non-null  int64         
 3   host_name                       91031 non-null  object        
 4   neighbourhood                   91031 non-null  object        
 5   latitude                        91031 non-null  float64       
 6   longitude                       91031 non-null  float64       
 7   room_type                       91031 non-null  object        
 8   price                           91031 non-null  float64       
 9   minimum_nights                  91031 non-null  int64         
 10  number_of_reviews               91031 non-null  int64         
 11  la

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,5396,Your Perfect Paris Studio On Île Saint-Louis,7903,Borzou,Hôtel-de-Ville,48.85247,2.35835,Entire home/apt,88.0,3,425,2024-11-23,2.26,1,77,51
1,7964,Sunny Apartment With Balcony,22155,Anaïs,Opéra,48.87417,2.34245,Entire home/apt,150.0,7,5,2015-09-14,0.03,1,0,0
2,9952,Paris Petit Coin Douillet,33534,Elisabeth,Popincourt,48.86238,2.36957,Entire home/apt,150.0,4,52,2024-09-24,0.36,1,250,3
3,11487,"Heart Of Paris, Brand New Aparment.",42666,Brigitte,Popincourt,48.86351,2.37110,Entire home/apt,80.0,30,13,2024-09-27,0.09,2,246,5
4,12452,Voltaire Charm,48733,Irene,Popincourt,48.85974,2.37932,Entire home/apt,201.0,1,62,2024-10-11,0.77,1,34,4
5,14264,Charming Sunny 2Br In Montmartre !,55995,Lara,Buttes-Montmartre,48.88259,2.34251,Entire home/apt,150.0,365,10,2020-03-14,0.07,1,180,0
6,17287,Unique Loft In Paris Near Parc De La Villette,64055,David,Buttes-Chaumont,48.89772,2.38508,Entire home/apt,82.0,4,48,2024-11-01,0.39,1,234,15
7,17994,Center Of Paris: Louvre,69389,Reem,Louvre,48.86185,2.34340,Entire home/apt,1200.0,4,202,2023-06-25,1.12,1,365,0
8,21004,Luxurious Penthouse+ 80 Sqm Terrace,79843,Nadia,Reuilly,48.83797,2.40058,Entire home/apt,200.0,2,129,2024-08-07,0.72,6,324,6
9,22158,Artist Loft With Rooftop In The City Center!,84874,Fabienne,Élysée,48.87039,2.32273,Private room,150.0,2,108,2024-10-18,0.74,1,86,14


## Résultats du nettoyage  
Maintenant  nous avons  un dataset bien nettoyé comportant   16 et on peut interpréter le jeu de données comme suit  :
Table représentant un séjour en appartement  Airbnb à Paris avec  des variables informant sur  :
* l'hôte
* les caractéristiques de  l'appartement
* Ses dimensions géospatiales
* Et ses  observations  (notes d'évaluation) après  le séjour.

Maintenant, nous  pouvons  passer à une 2ᵉ phase de notre  projet  de data  scieence  :
**E.D.A.**

# Analyse exploratoire  du  dataset (EDA)
pource faire  nous allons  travailler dans  un autre  fichier  jupyter  
Nous alons exporter  notre  table en csv  pour  l'utiliser   ensuite  

In [63]:
#Exportons  la table  nettoyée definitive en csv 
df.to_csv("df_clean", index = False)